<a href="https://colab.research.google.com/github/kilka-bez-hvosta/university_tasks/blob/main/AVL_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Реализуем наше AVL-Дерево, описав его структуру с помощью классов.

Первый шаг: узел.

In [ ]:
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None
        self.height = 1

    def display(self):
        lines, *_ = self._display_aux()
        for line in lines:
            print(line)

    def _display_aux(self): # собирает построчно все вершины
        # Если у вершины нет потомков - она вернет только одну строку содержающую эту вершину.
        if self.right is None and self.left is None:
            line = "%s" % self.value
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle
        # Если у вершины есть один из потомков, помимо самой вершины функция вернет и следующего потомка с учетом отступов.
        if self.right is None:
            lines, n, p, x = self.left._display_aux()
            s = "%s" % self.value
            u = len(s)
            first_line = (x + 1) * " " + (n - x - 1) * "_" + s
            second_line = x * " " + "/" + (n - x - 1 + u) * " "
            shifted_lines = [line + u * " " for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2
        if self.left is None:
            lines, n, p, x = self.right._display_aux()
            s = "%s" % self.value
            u = len(s)
            first_line = s + x * "_" + (n - x) * " "
            second_line = (u + x) * " " + "\\" + (n - x - 1) * " "
            shifted_lines = [u * " " + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2
        # Если у вершины оба потомка, то функция соберет результаты работы с их потомков, а затем сольет результаты с учетом отступов.
        left, n, p, x = self.left._display_aux()
        right, m, q, y = self.right._display_aux()
        s = "%s" % self.value
        u = len(s)
        first_line = (x + 1) * " " + (n - x - 1) * "_" + s + y * "_" + (m - y) * " "
        second_line = (
            x * " " + "/" + (n - x - 1 + u + y) * " " + "\\" + (m - y - 1) * " "
        )
        if p < q:
            left += [n * " "] * (q - p)
        elif q < p:
            right += [m * " "] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * " " + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2


Второй шаг: дерево

In [ ]:

class AVLTree:
    # Инициализация
    def __init__(self):
        self.root = None

    def height(self, node):
        if not node:
            return 0
        return node.height
    # Расчет баланса
    def balance(self, node):
        if not node:
            return 0
        return self.height(node.left) - self.height(node.right)
    # Вставка нового узла
    def insert(self, root, value):
        if not root:
            return Node(value) # если дерево пустое - вернем новый узел, который станет корнем
        elif value < root.value: # проверяем в какое поддерево попадет новый узел (рекурсивно отправляем его туда)
            root.left = self.insert(root.left, value)
        else:
            root.right = self.insert(root.right, value)


        root.height = 1 + max(self.height(root.left), self.height(root.right)) # меняем высоту каждого поддерева, в которое попали, т.к. он в пути нового узла
        balance = self.balance(root) # перерасчитываем балансировку

        # Левый поворот
        if balance > 1 and value < root.left.value:
            return self.right_rotate(root)

        # Правый поворот
        if balance < -1 and value > root.right.value:
            return self.left_rotate(root)

        # Большой поворот Лево-Право
        if balance > 1 and value > root.left.value:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        # Большой поворот Право-Лево
        if balance < -1 and value < root.right.value:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root
    # Удаление узла
    def delete(self, root, value):
        if not root: # нет дерева - нет узла
            return root
        if value < root.value: # рекурсивно ищем узел по условию
            root.left = self.delete(root.left, value)
        elif value > root.value:
            root.right = self.delete(root.right, value)
        else:
        # если одного из поддеревьев нет, то вершина существующего встанет на место удаляемого
            if not root.left:
                temp = root.right
                root = None
                return temp
            elif not root.right:
                temp = root.left
                root = None
                return temp

            temp = self.min_value_node(root.right)
            root.value = temp.value
            root.right = self.delete(root.right, temp.value)

        if not root:
            return root

        root.height = 1 + max(self.height(root.left), self.height(root.right)) # меняем высоту каждого поддерева в которое попали так как удалили кого-то из его потомков
        balance = self.balance(root) # перерасчет баланса

        # Левый поворот
        if balance > 1 and self.balance(root.left) >= 0:
            return self.right_rotate(root)

        # Правый поворот
        if balance < -1 and self.balance(root.right) <= 0:
            return self.left_rotate(root)

        # Большой Лево-Правый поворот
        if balance > 1 and self.balance(root.left) < 0:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        # Большой Право-Левый поворот
        if balance < -1 and self.balance(root.right) > 0:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root
    # Левый поворот
    def left_rotate(self, z):
        y = z.right
        T2 = y.left

        y.left = z
        z.right = T2

        z.height = 1 + max(self.height(z.left), self.height(z.right))
        y.height = 1 + max(self.height(y.left), self.height(y.right))

        return y

    def right_rotate(self, z):
        y = z.left
        T3 = y.right

        y.right = z
        z.left = T3

        z.height = 1 + max(self.height(z.left), self.height(z.right))
        y.height = 1 + max(self.height(y.left), self.height(y.right))

        return y

    def min_value_node(self, root):
        current = root
        while current.left:
            current = current.left
        return current

    def search(self, root, value):
        print(f"way: {root.value}")
        if not root or root.value == value:
            return root
        if root.value < value:
            return self.search(root.right, value)
        return self.search(root.left, value)

    def insert_value(self, value):
        self.root = self.insert(self.root, value)

    def delete_value(self, value):
        self.root = self.delete(self.root, value)

    def search_value(self, value):
        return self.search(self.root, value)




Шаг третий: примеры

Вставка:

In [ ]:
tree = AVLTree()
for i in range(10, 1000, 10):
  tree.insert_value(i)


tree.root.display()

                                                                                      ______________________________________________________________________________________________640______________________________________________                                                           
                                                                                     /                                                                                                                                               \                                                          
                                      ______________________________________________320______________________________________________                                                                         ______________________800______________________                                   
                                     /                                                                                               

Удаление:

In [ ]:
tree.delete_value(200)
print("Tree after deletion of 200:")
tree.root.display()

Tree after deletion of 200:
                                                                                   ______________________________________________________________________________________________640______________________________________________                                                           
                                                                                  /                                                                                                                                               \                                                          
                                      ___________________________________________320______________________________________________                                                                         ______________________800______________________                                   
                                     /                                                                            

Поиск значения:

In [ ]:
result = tree.search_value(30)
if result:
    print("Node found")
else:
    print("Node not found")

way: 640
way: 320
way: 160
way: 80
way: 40
way: 20
way: 30
Node found


Сортировка списка:

In [ ]:
from random import sample
array = sample(range(1000), 100)
print(array)

tree2 = AVLTree()
for i in array:
  tree2.insert_value(i)
tree2.root.display()
def sort_array(root, array):
        if root:
            if root.left:
                sort_array(root.left, array)
            array.append(root.value)
            if root.right:
                sort_array(root.right, array)
new_array = []
sort_array(tree2.root, new_array)
print(new_array)

[838, 522, 234, 619, 201, 93, 271, 226, 742, 471, 285, 759, 551, 465, 500, 693, 177, 501, 259, 862, 972, 91, 512, 778, 220, 135, 570, 227, 679, 768, 291, 387, 781, 630, 474, 80, 989, 597, 845, 66, 754, 718, 415, 542, 225, 362, 247, 242, 855, 212, 648, 897, 756, 746, 827, 75, 925, 641, 687, 36, 120, 384, 683, 164, 892, 546, 396, 76, 490, 341, 403, 769, 760, 355, 494, 979, 399, 277, 117, 481, 67, 81, 152, 813, 405, 43, 903, 535, 961, 817, 621, 869, 156, 713, 462, 607, 516, 930, 468, 420]
                                                             ________________________________________________________________471__________________________________________________________________________________                                                                                
                                                            /                                                                                                                                                     \       